#### Instal
l virtualenv
`virtualenv venv`

`source venv/bin/activate`

`pip install --upgrade pip`

`pip install -r requeriments.txt`


#### Deployment no GCS

`~/cursos/brazil-weather$ prefect deployment build ./el_prefect/el_flow.py:main -n flow-test -sb gcs/gcs-prefect/flow-test/ -q test`

Notar que `-sb gcs/xxx/yyy/` gcs é o protocolo e dpeois é as pastas no bucket

Depois fazer o apply do deployment:

`prefect deployment apply main-deployment.yaml`

Mesmo se o -sb for um bucket, o YAML fica no PC local

#### SSH na VM Linux GCP
- Gerar a chave [SSH Publica no Linux pessoal](https://cloud.google.com/compute/docs/connect/create-ssh-keys#create_an_ssh_key_pair):

`ssh-keygen -t rsa -f ~/.ssh/KEY_FILENAME -C USERNAME -b 2048`

- o -C vai o usuário da máquina linux, como eu logava com francis_romio, mantive isso....

- Ir em [Compute  Engine >> Metada >> SSH KEYS](https://console.cloud.google.com/compute/metadata)

- Fazer o upload da chave publica gerada.

- O IP externo fica nos detalhes da VM >> Network interfaces



##### Configurar Agent do Prefect

`prefect config set PREFECT_API_ENABLE_HTTP2=false`

https://github.com/PrefectHQ/prefect/issues/7442


##### BigQuery External Table

```sql
CREATE OR REPLACE EXTERNAL TABLE `br_weather.external_weather`
OPTIONS (
  format = 'PARQUET',
  uris = ['gs://zoomcamp-375723-prefect/data-lake/*.parquet']
);
```

#### Iniciar o PREFECT no uso

``` sh
prefect cloud login -k your-API-goes-here
prefect cloud workspace set --workspace $PREFECT_WORKSPACE
prefect agent start -q default
```

#### Deploy Final

```sh
prefect deployment build elt_flow.py:main_flow -n brazil-weather-flow -sb gcs/gcs-prefect -q default --cron "0 5 * * *" -o brazil-weather-flow.yaml
```

*Configurar Parameters no YAML do Flows*

```yaml
parameters:
  dict_param: {"BASE_URL": "https://portal.inmet.gov.br/uploads/dadoshistoricos/", "DEST_DIR": "./dump_zips/", "FILE_EXT": ".zip", "START_YEAR": 2023, "END_YEAR": 2023}
```

```yaml
parameters:
  dict_param:
    BASE_URL: https://portal.inmet.gov.br/uploads/dadoshistoricos/
    DEST_DIR: ./dump_zips/
    END_YEAR: 2023
    FILE_EXT: .zip
    START_YEAR: 2013
```

```sh
prefect deployment apply brazil-weather-flow.yaml
```